In [ ]:
# Runtime is about 3 hours

In [ ]:
import numpy as np
import os
import torch
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, Subset
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import transforms as T
from torchvision.transforms import ToTensor 
from utils.train import train_model,set_dataset,set_architecture
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_data1 = datasets.CIFAR10(
    root = 'data',
    train = True,                         
    transform = transform, 
    download = True,            
)
test_data1 = datasets.CIFAR10(
    root = 'data', 
    train = False, 
    transform = transform
)
train_data2 = datasets.CIFAR100(
    root = 'data',
    train = True,                         
    transform = transform, 
    download = True,            
)
test_data2 = datasets.CIFAR100(
    root = 'data', 
    train = False, 
    transform = transform
)


label_list=[0,8,16,70,22]
for i in range(50000):
  if train_data2.targets[i] not in label_list:
    train_data2.targets[i]=100
  if train_data2.targets[i]==0:
    train_data2.targets[i]=10
  elif train_data2.targets[i]==8:
    train_data2.targets[i]=11
  elif train_data2.targets[i]==16:
    train_data2.targets[i]=12
  elif train_data2.targets[i]==70:
    train_data2.targets[i]=13
  elif train_data2.targets[i]==22:
    train_data2.targets[i]=14

label_list=[0,8,16,70,22]
for i in range(10000):
  if test_data2.targets[i] not in label_list:
    test_data2.targets[i]=100
  if test_data2.targets[i]==0:
    test_data2.targets[i]=10
  elif test_data2.targets[i]==8:
    test_data2.targets[i]=11
  elif test_data2.targets[i]==16:
    test_data2.targets[i]=12
  elif test_data2.targets[i]==70:
    test_data2.targets[i]=13
  elif test_data2.targets[i]==22:
    test_data2.targets[i]=14

k=0
for i in range(50000):
  if train_data2.targets[i]!=100:
    if k==0:
      sub_data=train_data2.data[i]
      sub_data=sub_data.reshape(1,32,32,3)
      k=1
    else:
      sub_data=np.append(sub_data,train_data2.data[i].reshape(1,32,32,3),axis=0)

sub_target=[]
for i in range(50000):
  if train_data2.targets[i]!=100:
    sub_target.append(train_data2.targets[i])
train_data2.data=sub_data
train_data2.targets=sub_target

k=0
for i in range(10000):
  if test_data2.targets[i]!=100:
    if k==0:
      sub_data=test_data2.data[i]
      sub_data=sub_data.reshape(1,32,32,3)
      k=1
    else:
      sub_data=np.append(sub_data,test_data2.data[i].reshape(1,32,32,3),axis=0)
sub_target=[]
for i in range(10000):
  if test_data2.targets[i]!=100:
    sub_target.append(test_data2.targets[i])
test_data2.data=sub_data
test_data2.targets=sub_target

In [ ]:
train_data = torch.utils.data.ConcatDataset([train_data1, train_data2])
test_data = torch.utils.data.ConcatDataset([test_data1, test_data2])

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_data,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_data,
                batch_size=batch_size,
                shuffle=True)

In [ ]:
target_model = set_architecture('Resnet18',device, 15)
target_model = target_model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(target_model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
if not os.path.exists("./models/attack_model/DAA"):
    os.makedirs("./models/attack_model/DAA")
for epochs in range(2,5):
    model, acc = train_model(target_model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader, test_loader, 20*(epochs+1))
    torch.save(model,'./models/attack_model/DAA/DAA_'+str( 20*(epochs+1))+'.pt')
    acc_file=open('./models/attack_model/DAA/DAA_'+str( 20*(epochs+1))+'.txt','w')
    acc_file.write(str(acc))
    acc_file.close()